# __Modelos pre entrenados de Tensorflow Hub__

### Ajustar los datos en funcion del modelo a utilizar

Vamos a trabajar con las imagenes de que tenemos precargadas en local.

In [1]:
# librerias
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import string
import tensorflow as tf

from tensorflow.keras import regularizers

# libreria para trabajar con las bases de datos 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-11-13 10:32:00.499368: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 10:32:01.008793: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 10:32:01.008890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 10:32:01.173098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 10:32:01.582015: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 10:32:01.591037: I tensorflow/core/platform/cpu_feature_guard.cc:1

Creamos nuevos data generators, donde modificamos el target size y el color mode. Esto será aplicado a los 3 generadores.

In [2]:
# ruta de los archivos
train_dir = './sign-language-img/Train'
test_dir = './sign-language-img/Test'

# data generator
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

# generator para test, training y validation adaptados al modelo
train_generator_resize = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb',
    subset = 'training'
)

# para datos de validacion
validation_generator_resize = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb',
    subset = 'validation'
)

# para datos de prueba
test_generator_resize = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical',
    color_mode = 'rgb'
)

Found 27455 images belonging to 24 classes.
Found 1425 images belonging to 24 classes.
Found 7172 images belonging to 24 classes.


Definimos las clases de los datos

In [3]:
# clases
classes = [char for char in string.ascii_uppercase if char != 'J' if char != 'Z']

### Cargar modelo de Hub a utilizar 

Importamos la libreria

In [4]:
import tensorflow_hub as hub

Antes de todo, debemos detectar qué configuración usaremos, para esta ocasión usaremos la arquitectura [MobileNetV1](https://www.kaggle.com/models/google/mobilenet-v1/tensorFlow2/050-160-classification/1?tfhub-redirect=true).

Este modelo es secuencial, por lo que stackearemos sus capas en un modelo de este tipo.

Para los modelos del hub, basta con agregar una capa de entrada, añadir la capa de KerasLayer con la URL del sitio (no olvides configurarlo como no entrenable) y a partir de este punto podrás inyectar tu propia arquitectura.

In [5]:
model_url = 'https://tfhub.dev/google/imagenet/mobilenet_v1_050_160/classification/4'

model_hub = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(150, 150, 3)),
    hub.KerasLayer(model_url, trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

Antes de compilar la red debemos hacer build al modelo, siendo explícitos en las dimensiones de su tensor, en este caso MobileNet soporta frames de video, por lo que para usarlo como detector de imágenes bastará con anular esta dimensión

In [7]:
model_hub.build((None,150,150,3))
model_hub.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              1343049   
                                                                 
 flatten (Flatten)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 128)               128256    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 24)                3096      
                                                                 
Total params: 1474401 (5.62 MB)
Trainable params: 131352 (513.09 KB)
Non-trainable params: 1343049 (5.12 MB)
_________________________________________________________________


Podemos notar en la arquitectura que se añade toda la capa cargada y posteriormente nuestra arquitectura.

El proceso de compilación y entrenamiento será el tradicional.

In [8]:
model_hub.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
    )

In [10]:
history_hub = model_hub.fit(
    train_generator_resize,
    epochs=2,
    validation_data=validation_generator_resize
)

Epoch 1/2


2024-11-13 10:36:50.113740: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2024-11-13 10:36:58.672016: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2024-11-13 10:36:59.412676: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 34572288 bytes after encountering the first element of size 34572288 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-13 10:36:59.429208: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46080000 exceeds 10% of free system memory.
2024-11-13 10:37:00.377389: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 46080000 exceeds 10% of free system memory.
2024-11-13 10:37:00.747956: W external/local_tsl/tsl/framework/cpu_all

215/215 [==============================] - 267s 1s/step - loss: 1.5760 - accuracy: 0.5150 - val_loss: 0.8558 - val_accuracy: 0.6989
Epoch 2/2
215/215 [==============================] - 235s 1s/step - loss: 0.6649 - accuracy: 0.7804 - val_loss: 0.6025 - val_accuracy: 0.7832


Este tipo de modelos son útiles a la hora de procesar imágenes en vivo (como en cámaras de drones). Pero traen en consecuencias una pérdida de precisión, dependerá de tu proyecto aceptar estas alternativas o sacrificar tiempo de cómputo por mejores resultados.

Evaluemos el modelo cn los datos de prueba

In [11]:
results = model_hub.evaluate(test_generator_resize)

57/57 [==============================] - 62s 1s/step - loss: 0.6063 - accuracy: 0.7885


In [12]:
results

[0.6063175797462463, 0.7884829640388489]

> tenemos una precision muy baja respecto al modelo anterior y una perdida alta